In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from fbprophet import Prophet

In [2]:
# 导入数据
sales = pd.read_csv("../data/sales_train.csv")
test = pd.read_csv("../data/test.csv")
item_cat = pd.read_csv("../data/item_categories.csv")
item = pd.read_csv("../data/items.csv")
sub = pd.read_csv("../data/sample_submission.csv")
shops = pd.read_csv("../data/shops.csv")

### 异常值处理

In [3]:
sales = sales[(sales.item_price < 100000)&(sales.item_cnt_day < 1001)]

median = sales[(sales.shop_id==32)&(sales.item_id==2973)&(sales.date_block_num==4)&
              (sales.item_price > 0)].item_price.median()
sales.loc[sales.item_price<=0, 'item_price'] = median

#### 按月销量进行汇总，NaN替换成0

In [4]:
monthly_sales=sales.groupby(["shop_id","item_id","date_block_num"])["item_cnt_day"].sum()
monthly_sales=monthly_sales.unstack(level=-1).fillna(0)
monthly_sales=monthly_sales.T
dates=pd.date_range(start = '2013-01-01',end='2015-10-01', freq = 'MS')
monthly_sales.index=dates
monthly_sales.head()

shop_id       0                                                         ...   \
item_id    30    31    32    33    35    36    40    42    43    49     ...    
2013-01-01   0.0   0.0   6.0   3.0   1.0   0.0   0.0   0.0   1.0   0.0  ...    
2013-02-01  31.0  11.0  10.0   3.0  14.0   1.0   1.0   1.0   0.0   2.0  ...    
2013-03-01   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...    
2013-04-01   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...    
2013-05-01   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...    

shop_id       59                                                        
item_id    22144 22145 22150 22151 22153 22154 22155 22162 22164 22167  
2013-01-01   0.0   0.0   0.0   2.0   0.0   1.0   0.0   0.0   0.0   0.0  
2013-02-01   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
2013-03-01   0.0   0.0   0.0   1.0   0.0   0.0   0.0   0.0   0.0   0.0  
2013-04-01   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
2013-05-01   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  

[5 rows x 424123 columns]

### 模型训练

In [5]:
start_time=time.time()

forecastsDict = {}
for i in range(len(test)):
    shop_id = test.iloc[i]['shop_id']
    item_id = test.iloc[i]['item_id']
    
    train_shop = sales[sales['shop_id'] == shop_id]
    train_item = train_shop[train_shop['item_id'] == item_id]

    if not train_item.empty : 
        nodeToForecast = pd.DataFrame(monthly_sales[shop_id][item_id])
        nodeToForecast.reset_index(inplace=True)
        nodeToForecast = nodeToForecast.rename(columns = {nodeToForecast.columns[0] : 'ds',nodeToForecast.columns[1] : 'y'})
        
        growth = 'linear'
        m = Prophet(growth, yearly_seasonality=True)
        m.fit(nodeToForecast)
        future = m.make_future_dataframe(periods = 1, freq = 'MS')
        forecast = m.predict(future)
        yhat =  forecast['yhat'].iloc[-1]
        if yhat < 0:
            yhat = 0
        forecastsDict[i] =  yhat
    else:
        forecastsDict[i] = 0
    print('#'+ str(i)+': '+ str(forecastsDict[i]))
end_time = time.time()

INFO:fbprophet.forecaster:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
C:\Users\yangman\Anaconda3\lib\site-packages\pystan\misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):
INFO:fbprophet.forecaster:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


#0: 1.1945781707626533
#1: 0


INFO:fbprophet.forecaster:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


#2: 0.6031885395403548


INFO:fbprophet.forecaster:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


...

#99: 2.3711415951974613


INFO:fbprophet.forecaster:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


#100: 31.438624529660235


INFO:fbprophet.forecaster:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


#101: 0


In [ ]:
submission = pd.DataFrame({'ID': test['ID'], 'item_cnt_month': forecastsDict})
submission.to_csv('./submission.csv', index=False,sep=',')